In [7]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values='NA')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values='NA')
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values='NA', schema_overrides={'precip': pl.Float64, 'visib': pl.Float64})


flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))


# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


In [15]:
#Q1.1
result = ctx.execute("""
    SELECT DISTINCT carrier
    FROM airlines
    ORDER BY carrier
""")

print(result)


shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


In [16]:
#Q1.2
result = ctx.execute("""
    SELECT
        dest,
        COUNT(*) AS num_flights
    FROM flights
    GROUP BY dest
    ORDER BY num_flights DESC
    LIMIT 10
""")

print(result)


shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


In [19]:
#Q1.3
result = ctx.execute("""
    SELECT
        year, month, day,
        carrier, flight, tailnum,
        origin, dest,
        dep_time, dep_delay, sched_dep_time,
        time_hour
    FROM flights
    WHERE dep_delay > 120
    ORDER BY dep_delay DESC
""")

print(result)

shape: (9_723, 12)
┌──────┬───────┬─────┬─────────┬───┬──────────┬───────────┬────────────────┬───────────────────────┐
│ year ┆ month ┆ day ┆ carrier ┆ … ┆ dep_time ┆ dep_delay ┆ sched_dep_time ┆ time_hour             │
│ ---  ┆ ---   ┆ --- ┆ ---     ┆   ┆ ---      ┆ ---       ┆ ---            ┆ ---                   │
│ i64  ┆ i64   ┆ i64 ┆ str     ┆   ┆ i64      ┆ i64       ┆ i64            ┆ datetime[μs, UTC]     │
╞══════╪═══════╪═════╪═════════╪═══╪══════════╪═══════════╪════════════════╪═══════════════════════╡
│ 2013 ┆ 1     ┆ 9   ┆ HA      ┆ … ┆ 641      ┆ 1301      ┆ 900            ┆ 2013-01-09 14:00:00   │
│      ┆       ┆     ┆         ┆   ┆          ┆           ┆                ┆ UTC                   │
│ 2013 ┆ 6     ┆ 15  ┆ MQ      ┆ … ┆ 1432     ┆ 1137      ┆ 1935           ┆ 2013-06-15 23:00:00   │
│      ┆       ┆     ┆         ┆   ┆          ┆           ┆                ┆ UTC                   │
│ 2013 ┆ 1     ┆ 10  ┆ MQ      ┆ … ┆ 1121     ┆ 1126      ┆ 1635        

In [20]:
#Q2.1
result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) AS avg_dep_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_dep_delay DESC
""")

print(result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘


In [21]:
#Q2.2
result = ctx.execute("""
    SELECT
        month,
        COUNT(*) AS num_flights
    FROM flights
    GROUP BY month
    ORDER BY num_flights DESC
    LIMIT 1
""")

print(result)

shape: (1, 2)
┌───────┬─────────────┐
│ month ┆ num_flights │
│ ---   ┆ ---         │
│ i64   ┆ u32         │
╞═══════╪═════════════╡
│ 7     ┆ 29425       │
└───────┴─────────────┘


In [22]:
#Q3.1

result = ctx.execute("""
    SELECT
        f.carrier,
        a.name        AS airline_name,
        f.flight,
        f.origin,
        f.dest
    FROM flights AS f
    JOIN airlines AS a
      ON f.carrier = a.carrier
    ORDER BY f.time_hour, f.carrier, f.flight
    LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1696   ┆ EWR    ┆ ORD  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 71     ┆ JFK    ┆ TPA  │
│ B6      ┆ JetBlue Airways        ┆ 79     ┆ JFK    ┆ MCO  │
│ B6      ┆ JetBlue Airways        ┆ 102    ┆ JFK    ┆ BUF  │
│ B6      ┆ JetBlue Airways        ┆ 117    ┆ JFK    ┆ MSY  │
│ B6      ┆ JetBlue Airways        ┆ 125    ┆ JFK    ┆ 

In [23]:
#Q3.2

result = ctx.execute("""
    SELECT
        f.carrier,
        AVG(2013 - p.year) AS avg_aircraft_age
    FROM flights AS f
    JOIN planes  AS p
      ON f.tailnum = p.tailnum
    WHERE p.year IS NOT NULL
      AND 2013 - p.year >= 0
    GROUP BY f.carrier
    ORDER BY avg_aircraft_age DESC
""")

print(result)


shape: (16, 2)
┌─────────┬──────────────────┐
│ carrier ┆ avg_aircraft_age │
│ ---     ┆ ---              │
│ str     ┆ f64              │
╞═════════╪══════════════════╡
│ MQ      ┆ 35.319           │
│ AA      ┆ 25.869426        │
│ DL      ┆ 16.372169        │
│ UA      ┆ 13.207691        │
│ FL      ┆ 11.385829        │
│ …       ┆ …                │
│ B6      ┆ 6.686702         │
│ F9      ┆ 4.87874          │
│ VX      ┆ 4.473643         │
│ AS      ┆ 3.33662          │
│ HA      ┆ 1.548387         │
└─────────┴──────────────────┘


In [24]:
#Q4.1

result = ctx.execute("""
    SELECT
        COALESCE(p.manufacturer, 'Unknown') AS manufacturer,
        COALESCE(p.model,        'Unknown') AS model,
        COUNT(*) AS num_flights
    FROM flights AS f
    LEFT JOIN planes AS p
      ON f.tailnum = p.tailnum
    GROUP BY manufacturer, model
    ORDER BY num_flights DESC
    LIMIT 10
""")

print(result)

shape: (10, 3)
┌──────────────────┬─────────────────┬─────────────┐
│ manufacturer     ┆ model           ┆ num_flights │
│ ---              ┆ ---             ┆ ---         │
│ str              ┆ str             ┆ u32         │
╞══════════════════╪═════════════════╪═════════════╡
│ Unknown          ┆ Unknown         ┆ 52606       │
│ AIRBUS           ┆ A320-232        ┆ 31278       │
│ EMBRAER          ┆ EMB-145LR       ┆ 28027       │
│ EMBRAER          ┆ ERJ 190-100 IGW ┆ 23716       │
│ AIRBUS INDUSTRIE ┆ A320-232        ┆ 14553       │
│ EMBRAER          ┆ EMB-145XR       ┆ 14051       │
│ BOEING           ┆ 737-824         ┆ 13809       │
│ BOMBARDIER INC   ┆ CL-600-2D24     ┆ 11807       │
│ BOEING           ┆ 737-7H4         ┆ 10389       │
│ BOEING           ┆ 757-222         ┆ 9150        │
└──────────────────┴─────────────────┴─────────────┘


In [25]:
#Q4.2

result = ctx.execute("""
    WITH route AS (
        SELECT
            f.origin,
            f.dest,
            COUNT(*) AS total_flights,
            AVG(f.dep_delay) AS avg_dep_delay,
            CAST(SUM(CASE WHEN f.dep_delay > 30 THEN 1 ELSE 0 END) AS DOUBLE)
              / NULLIF(COUNT(*), 0) AS pct_delay_gt_30
        FROM flights AS f
        WHERE f.dep_delay IS NOT NULL
        GROUP BY f.origin, f.dest
    )
    SELECT
        r.origin,
        ao.name AS origin_name,
        r.dest,
        ad.name AS dest_name,
        r.total_flights,
        r.avg_dep_delay,
        r.pct_delay_gt_30
    FROM route AS r
    LEFT JOIN airports AS ao
      ON r.origin = ao.faa
    LEFT JOIN airports AS ad
      ON r.dest = ad.faa
    ORDER BY r.total_flights DESC
    LIMIT 10
""")

print(result)

shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ pct_delay_gt_3 │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ 0              │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11196         ┆ 8.522508      ┆ 0.098875       │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆               ┆                │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10082         ┆ 11.448621     ┆ 0.124678       │
│        ┆                ┆      ┆ Jackson        ┆               ┆         

In [26]:
#Bonus Question

polars_top_dest = (
    flights
    .group_by("dest")
    .agg(pl.len().alias("num_flights"))
    .sort("num_flights", descending=True)
    .limit(10)
)

print(polars_top_dest)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘
